In [279]:
:dep ratatui
:dep ratatui-macros

Ratatui is a crate for building terminal user interfaces in Rust.

One of the unique features of Ratatui is that it is an immediate mode rendering library.
In this post, I'm going to describe some of the primitives of Ratatui.
I rely on the concepts described in this post in every Ratatui application I build.

## Immediate Mode Rendering

User interfaces can broadly be classified into two kinds:

- immediate mode GUIs,
- retained mode GUIs.

Casey Muratori has a great video on immediate mode rendering. 

{{< video https://www.youtube.com/watch?v=Z1qyvQsjK5Y }}

At a very high level, in retained mode GUIs, you create UI elements and pass it to a framework and the framework is in charge of displaying them. For example, you can create a text field and input field, and then the browser will render them. The browser is in charge of handling events, and as a developer you have to define how these events interact with these widgets.

In [280]:
fn show_html<D>(content: D) where D: std::fmt::Display {
    println!(r#"EVCXR_BEGIN_CONTENT text/html
<div style="display: flex; justify-content:start; gap: 1em">
{}
</div>
EVCXR_END_CONTENT"#, content);
}

For example, in a simple counter example in a browser, we have to set up an `incrementCounter` and `decrementCounter` callbacks that update the relevant element's state.

In [281]:
show_html(r#"
<text> Counter: </text>
<text id="counter">0</text>

<button onclick="incrementCounter()">Increment</button>
<button onclick="decrementCounter()">Decrement</button>

<script>
    var counterElement = document.getElementById("counter");

    var counterValue = 0;
    counterElement.textContent = counterValue;

    function incrementCounter() {
        counterValue++;
        counterElement.textContent = counterValue;
    }

    function decrementCounter() {
        counterValue--;
        counterElement.textContent = counterValue;
    }
</script>
"#)

()

In immediate mode rendering, you are drawing the UI every "frame" in a for loop.
Let's say you have the following `App`:

In [282]:
#[derive(Debug, Default)]
pub struct App {
    counter: u8,
}
let mut app = App::default();
app

App { counter: 0 }

And you are rendering into an 80 wide, 5 tall terminal: 

In [283]:
let area = Rect::new(0, 0, 80, 5);
let mut buf = Buffer::empty(area);
buf

Buffer {
    area: Rect { x: 0, y: 0, width: 80, height: 5 },
    content: [
        "                                                                                ",
        "                                                                                ",
        "                                                                                ",
        "                                                                                ",
        "                                                                                ",
    ],
    styles: [
        x: 0, y: 0, fg: Reset, bg: Reset, underline: Reset, modifier: NONE,
    ]
}

Let's import some useful widgets and building blocks:

In [284]:
use ratatui::widgets::Block;

We can render into the buffer once by creating a `Block` with a border and rendering into an area that is a subset of the buffer.

In [285]:
let block = Block::bordered();
block.render(area, &mut buf);

In [286]:
buf

Buffer {
    area: Rect { x: 0, y: 0, width: 80, height: 5 },
    content: [
        "┌──────────────────────────────────────────────────────────────────────────────┐",
        "│                                                                              │",
        "│                                                                              │",
        "│                                                                              │",
        "└──────────────────────────────────────────────────────────────────────────────┘",
    ],
    styles: [
        x: 0, y: 0, fg: Reset, bg: Reset, underline: Reset, modifier: NONE,
    ]
}

In [287]:
let block = Block::bordered().title("Counter Example");
block.render(area, &mut buf);

In [288]:
buf

Buffer {
    area: Rect { x: 0, y: 0, width: 80, height: 5 },
    content: [
        "┌Counter Example───────────────────────────────────────────────────────────────┐",
        "│                                                                              │",
        "│                                                                              │",
        "│                                                                              │",
        "└──────────────────────────────────────────────────────────────────────────────┘",
    ],
    styles: [
        x: 0, y: 0, fg: Reset, bg: Reset, underline: Reset, modifier: NONE,
    ]
}

Now let's render put text into a `Paragraph` widget and render that into the `Buffer`.

In [289]:
let text = Text::from(vec![
    "".into(),
    format!("Counter: {}", app.counter).into(),
]);
let block = Block::bordered().title("Counter Example");

let paragraph = Paragraph::new(text).block(block).centered();
paragraph.render(area, &mut buf);

buf

Buffer {
    area: Rect { x: 0, y: 0, width: 80, height: 5 },
    content: [
        "┌Counter Example───────────────────────────────────────────────────────────────┐",
        "│                                                                              │",
        "│                                  Counter: 0                                  │",
        "│                                                                              │",
        "└──────────────────────────────────────────────────────────────────────────────┘",
    ],
    styles: [
        x: 0, y: 0, fg: Reset, bg: Reset, underline: Reset, modifier: NONE,
    ]
}

This is one frame of our UI!

Let's put our UI code into a function.

In [290]:
fn draw_ui(app: &App, area: Rect, buf: &mut Buffer) {
    let text = Text::from(vec![
        "".into(),
        format!("Counter: {}", app.counter).into(),
    ]);
    let block = Block::bordered().title("Counter Example");
    
    let paragraph = Paragraph::new(text).block(block).centered();
    paragraph.render(area, buf);
}

For the next frame, we can increment the counter and render into the buffer again.

In [291]:
app.counter += 1;

draw_ui(&app, area, &mut buf);

buf

Buffer {
    area: Rect { x: 0, y: 0, width: 80, height: 5 },
    content: [
        "┌Counter Example───────────────────────────────────────────────────────────────┐",
        "│                                                                              │",
        "│                                  Counter: 1                                  │",
        "│                                                                              │",
        "└──────────────────────────────────────────────────────────────────────────────┘",
    ],
    styles: [
        x: 0, y: 0, fg: Reset, bg: Reset, underline: Reset, modifier: NONE,
    ]
}

If we repeat this process of "updating state" and "drawing UI" in a loop, we get an immediate mode rendered UI.

Here's what a more complete counter application might look like with keyboard events.

![](./basic-app.webp)

If you are interested in seeing the full code regarding this, you can check out the [`basic-app`] tutorial on the Ratatui website.

[`basic-app`]: https://ratatui.rs/tutorials/counter-app/basic-app/


## Text primitives

In Ratatui, there are 3 fundamental text primitives that you should be aware of.

### Span

The first is a `Span`.

In [292]:
use ratatui::text::Span;
let span = Span::raw("hello world");
span

Span { content: "hello world", style: Style { fg: None, bg: None, underline_color: None, add_modifier: NONE, sub_modifier: NONE } }

A `Span` contains two fields. 

1. `content`

In [293]:
span.content

"hello world"

2. `style`

In [294]:
span.style

Style { fg: None, bg: None, underline_color: None, add_modifier: NONE, sub_modifier: NONE }

A `Style` object contains foreground color, background color, and modifiers for whether the style being applied is **bold**, _italics_, etc

There are a number of constructors for `Span`, but `ratatui` exposes a trait that makes it easy to convert any String into a styled span.

In [295]:
use ratatui::style::Stylize;
"hello world".bold()

Span { content: "hello world", style: Style { fg: None, bg: None, underline_color: None, add_modifier: BOLD, sub_modifier: NONE } }

You can even chain these trait methods to add more styles:

In [296]:
"hello world".bold().yellow().on_black()

Span { content: "hello world", style: Style { fg: Some(Yellow), bg: Some(Black), underline_color: None, add_modifier: BOLD, sub_modifier: NONE } }

In [297]:
fn show_span(s: Span) {
    let mut html = String::new();
    html.push_str("<span style=\"");

    // Set foreground color
    if let Some(color) = &s.style.fg {
        html.push_str(&format!("color: {};", color));
    }

    // Set background color
    if let Some(color) = &s.style.bg {
        html.push_str(&format!("background-color: {};", color));
    }

    // Add modifiers
    match s.style.add_modifier {
        Modifier::BOLD => html.push_str("font-weight: bold;"),
        Modifier::UNDERLINED => html.push_str("text-decoration: underline;"),
        _ => {}
    }
    html.push_str("\">");
    html.push_str(&s.content);
    html.push_str("</span>");
    show_html(html)
}

In [298]:
show_span("hello world".bold())

()

In [299]:
show_span("hello world".yellow().bold().on_black())

()

With `ratatui-macros`, you can even use a `format!` style macro to create a `Span`

In [300]:
use ratatui_macros::span;

let world = "world";
span!("hello {}", world)

Span { content: "hello world", style: Style { fg: None, bg: None, underline_color: None, add_modifier: NONE, sub_modifier: NONE } }

### Line

The second primitive to be aware of is a `Line`.

A line consists of one or more spans.

In [301]:
let line = Line::raw("hello world");
line

Line { spans: [Span { content: "hello world", style: Style { fg: None, bg: None, underline_color: None, add_modifier: NONE, sub_modifier: NONE } }], style: Style { fg: None, bg: None, underline_color: None, add_modifier: NONE, sub_modifier: NONE }, alignment: None }

A unique feature of lines is that new lines are removed but the content is split into multiple spans. 

In [302]:
let line = Line::raw("hello world\ngoodbye world");
line.spans.len()

2

Using a newline is equivalent to doing the following:

In [303]:
let line = Line::default().spans(vec![Span::raw("hello world"), Span::raw("goodbye world")]);
line.spans.len()

2

A line can also be styled with methods from the `Stylize` trait:

In this case, the individual span's styles are left untouched but the `Line`'s style is updated.

Another unique feature about `Line` is that they can be aligned.

In [304]:
let centered_line = line.centered();
centered_line.alignment

Some(Center)

With `ratatui-macros`, you can create a `Line` using the `line!` macro using a `vec!`-like syntax.

In [305]:
use ratatui_macros::line;

line!["hello", " ", "world"].yellow().bold().centered();

### Text

Finally there is `Text`, which is a collection of `Line`s.

In [306]:
Text::from(vec![Line::raw("hello world"), Line::raw("goodbye world")]);

With ratatui-macros, you can create a `Text` using `text!` macro using a `vec!`-like syntax.

In [307]:
text!["hello world", "goodbye world"];

Like `Line`, `Text` can also be aligned. In this case, the alignment occurs on every `Line` inside the `Text`.

In [308]:
let t = text!["hello world", "goodbye world"].right_aligned();
t.alignment

Some(Right)

Earlier, we created a `Text` element like this:

In [310]:
let text = Text::from(vec![
    "".into(),
    format!("Counter: {}", app.counter).into(),
]);

This can be simplified like this:

In [311]:
text!["", format!("Counter: {}", app.counter)];

## Widget primitives

## Block

The simplest widget is the `Block` widget, which is essentially just borders.

In [321]:
let (x, y, width, height) = (0, 0, 50, 5); 
let area = Rect::new(x, y, width, height);
let mut buf = Buffer::empty(area);
Block::bordered().render(area, &mut buf);
buf

Buffer {
    area: Rect { x: 0, y: 0, width: 50, height: 5 },
    content: [
        "┌────────────────────────────────────────────────┐",
        "│                                                │",
        "│                                                │",
        "│                                                │",
        "└────────────────────────────────────────────────┘",
    ],
    styles: [
        x: 0, y: 0, fg: Reset, bg: Reset, underline: Reset, modifier: NONE,
    ]
}

Most widgets accept a `Block` as a fluent setter. We saw from earlier that the `Paragraph` has a `.block()` method that accepts a `Block`.

```rust
let paragraph = Paragraph::new(text).block(block).centered();
```

Blocks can have different kinds of borders:

In [333]:
let (x, y, width, height) = (0, 0, 50, 5); 
let area = Rect::new(x, y, width, height);
let mut buf = Buffer::empty(area);

Block::bordered().border_type(ratatui::widgets::BorderType::Double).render(area.inner(&Margin::new(2, 1)), &mut buf);
Block::bordered().borders(ratatui::widgets::Borders::TOP | ratatui::widgets::Borders::BOTTOM).render(area, &mut buf);

buf

Buffer {
    area: Rect { x: 0, y: 0, width: 50, height: 5 },
    content: [
        "──────────────────────────────────────────────────",
        "  ╔════════════════════════════════════════════╗  ",
        "  ║                                            ║  ",
        "  ╚════════════════════════════════════════════╝  ",
        "──────────────────────────────────────────────────",
    ],
    styles: [
        x: 0, y: 0, fg: Reset, bg: Reset, underline: Reset, modifier: NONE,
    ]
}

And `Block` can have multiple titles in different locations:

In [347]:
let (x, y, width, height) = (0, 0, 50, 5); 
let area = Rect::new(x, y, width, height);
let mut buf = Buffer::empty(area);

let block = Block::bordered()
                .title("Top Left") // accepts anything that can be converted to a `Title` or a `Line`
                .title(Line::from("Top Center").centered()) // explicitly need to use `Line` if you want alignment
                .title(line!["Top Right"].right_aligned()) // you can use the `line!` macro to make it shorter
                .title(ratatui::widgets::block::Title::from("Bottom Right") // explicitly using `Title` gives you most control
                       .alignment(ratatui::layout::Alignment::Right)
                       .position(ratatui::widgets::block::title::Position::Bottom)
                )
                .title_bottom(Line::from("Bottom Center").centered()) // shorthand functions for bottom position
                .title_bottom("Bottom Left"); // aligned to the left by default

block.render(area, &mut buf);

buf

Buffer {
    area: Rect { x: 0, y: 0, width: 50, height: 5 },
    content: [
        "┌Top Left───────────Top Center──────────Top Right┐",
        "│                                                │",
        "│                                                │",
        "│                                                │",
        "└Bottom Left──────Bottom Center──────Bottom Right┘",
    ],
    styles: [
        x: 0, y: 0, fg: Reset, bg: Reset, underline: Reset, modifier: NONE,
    ]
}

## Conclusion

In the next post, we'll examine the other widgets and how Ratatui works under the hood in more detail.